In [12]:
import os

import numpy as np
from keras import Sequential, Model
from keras.layers import Dense
from keras.optimizer_v2.adam import Adam

from src.dlla.hw import encode, dlla_hw
from src.dlla.wegener import make_mlp_wegener, binomial_test
from src.pollution.clock_jitter import clock_jitter
from src.pollution.gaussian_noise import gaussian_noise
from src.pollution.random_delay import random_delay
from src.trace_set.database import Database
from src.trace_set.pollution import Pollution, PollutionType
from src.trace_set.set_hw import TraceSetHW
from src.trace_set.transform import reduce_fixed_fixed, fixed_fixed
from src.trace_set.window import get_windows
from src.tvla.cri import tvla_cri
from src.tvla.tvla import Group

In [2]:
DB = Database.aisy

In [3]:
TRACE_SET = TraceSetHW(DB)

X_PROF_CXT, Y_PROF = TRACE_SET.profile()
X_ATT_CXT, Y_ATT = TRACE_SET.attack()

A_THIRD = round(X_PROF_CXT.shape[1] / 3)
SAMPLE_TRACE = X_PROF_CXT[0][A_THIRD:A_THIRD*2]

In [4]:
WINDOW, _ = get_windows(X_PROF_CXT, SAMPLE_TRACE)

In [5]:
X_PROF = X_PROF_CXT[:, WINDOW[0]:WINDOW[1]]
X_ATT = X_ATT_CXT[:, WINDOW[0]:WINDOW[1]]

X_PROF_CXT.shape, X_PROF.shape, Y_PROF.shape

((6667, 7332), (6667, 2444), (6667,))

In [6]:
JITTER_PARAMS = np.arange(0, 42, 4)

def apply_jitter(params):
    for param in params:
        pollution = Pollution(PollutionType.jitter, param)
        out = TraceSetHW(DB, pollution)

        if not os.path.exists(out.path):
            x = clock_jitter(X_PROF_CXT, WINDOW, param)
            x_att = clock_jitter(X_ATT_CXT, WINDOW, param)

            out.create(x, Y_PROF, x_att, Y_ATT)

            x2, y2 = fixed_fixed(x, Y_PROF)
            a, b = x2[~y2], x2[y2]
            order = 2
            _, tvla_p = Group(a, order, True).t_test(Group(b, order, True), order)
            print(f"Clock jitter ({param}): min-p: ({min(tvla_p)}).")

apply_jitter(JITTER_PARAMS)

Clock jitter (4):   0%|          | 28/6667 [00:00<00:24, 272.31it/s]

Clock jitter (0): min-p: (0.0).


Clock jitter (8):   1%|          | 45/6667 [00:00<00:14, 444.06it/s]

Clock jitter (4): min-p: (8.068355586708169e-266).


Clock jitter (12):   1%|          | 61/6667 [00:00<00:10, 606.58it/s]

...

Clock jitter (32): min-p: (6.589274128741541e-50).


Clock jitter (40):   3%|▎         | 168/6667 [00:00<00:03, 1672.03it/s]

Clock jitter (36): min-p: (3.038727949337326e-63).


Computing Central Moments: 100%|██████████| 4/4 [00:00<00:00, 60.41it/s]

Clock jitter (40): min-p: (2.8437025283860074e-54).


In [13]:
GAUSS_PARAMS = np.arange(0, 32, 2)

def apply_gauss(params):
    for param in params:
        pollution = Pollution(PollutionType.gauss, param)
        out = TraceSetHW(DB, pollution)

        if not os.path.exists(out.path):
            x = gaussian_noise(X_PROF, param)
            x_att = gaussian_noise(X_ATT, param)

            out.create(x, Y_PROF, x_att, Y_ATT)

            x2, y2 = fixed_fixed(x, Y_PROF)
            a, b = x2[~y2], x2[y2]
            order = 2
            _, tvla_p = Group(a, order, True).t_test(Group(b, order, True), order)
            print(f"Gaussian noise ({param}): min-p: ({min(tvla_p)}).")

apply_gauss(GAUSS_PARAMS)

Computing Central Moments: 100%|██████████| 4/4 [00:00<00:00, 60.56it/s]


Gaussian noise (0): min-p: (0.0).


Computing Central Moments: 100%|██████████| 4/4 [00:00<00:00, 62.10it/s]


Gaussian noise (2): min-p: (0.0).


Computing Central Moments: 100%|██████████| 4/4 [00:00<00:00, 54.92it/s]


...

Gaussian noise (26): min-p: (9.40259986232613e-19).


Computing Central Moments: 100%|██████████| 4/4 [00:00<00:00, 58.02it/s]


Gaussian noise (28): min-p: (7.759068597738775e-18).


Computing Central Moments: 100%|██████████| 4/4 [00:00<00:00, 54.47it/s]

Gaussian noise (30): min-p: (3.016352289026674e-16).


In [14]:
DELAY_PARAMS = np.arange(0, .80, .05)
A = 5
B = 3
DELAY_AMP = 10

def apply_delay(params):
    for param in params:
        pollution = Pollution(PollutionType.delay, param)
        out = TraceSetHW(DB, pollution)

        if not os.path.exists(out.path):
            x = random_delay(X_PROF, A, B, DELAY_AMP, param)
            x_att = random_delay(X_ATT, A, B, DELAY_AMP, param)

            out.create(x, Y_PROF, x_att, Y_ATT)

            x2, y2 = fixed_fixed(x, Y_PROF)
            a, b = x2[~y2], x2[y2]
            order = 2
            _, tvla_p = Group(a, order, True).t_test(Group(b, order, True), order)
            print(f"Random delay ({param}): min-p: ({min(tvla_p)}).")

apply_delay(DELAY_PARAMS)

Random delay (0.05):   1%|▏         | 92/6667 [00:00<00:07, 911.11it/s]

Random delay (0.0): min-p: (0.0).


Random delay (0.1):   1%|          | 76/6667 [00:00<00:08, 758.40it/s]

Random delay (0.05): min-p: (0.0).


Random delay (0.15000000000000002):   1%|          | 59/6667 [00:00<00:11, 581.31it/s]

...

Random delay (0.65): min-p: (0.0).


Random delay (0.75):   1%|          | 47/6667 [00:00<00:14, 468.56it/s]

Random delay (0.7000000000000001): min-p: (2.3357431488435326e-273).


Computing Central Moments: 100%|██████████| 4/4 [00:00<00:00, 62.03it/s]

Random delay (0.75): min-p: (1.5659863412983572e-251).


In [7]:
DELAY_PARAMS = np.arange(0, .5, .05)
A = 5
B = 3
DELAY_AMP = 10

def apply_delay(params):
    for param in params:
        pollution = Pollution(PollutionType.delay, param)
        out = TraceSetHW(DB, pollution)

        if not os.path.exists(out.path):
            x = random_delay(X_PROF, A, B, DELAY_AMP, param)
            x_att = random_delay(X_ATT, A, B, DELAY_AMP, param)

            out.create(x, Y_PROF, x_att, Y_ATT)

            x2, y2 = fixed_fixed(x, Y_PROF)
            a, b = x2[~y2], x2[y2]
            order = 2
            _, tvla_p = Group(a, order, True).t_test(Group(b, order, True), order)
            print(f"Random delay ({param}): min-p: ({min(tvla_p)}).")

# apply_delay(DELAY_PARAMS)

In [8]:
GAUSS_PARAMS = np.arange(0, 32, 2)

### TVLA vs. DL-LA

In [9]:
ORDER = 2

def store_results(database: Database, method: str, pollution: Pollution, p):
    file_name = f"results_{database.name}.csv"
    with open(file_name, 'a') as f:
        f.write(f"{method};{pollution.type.name};{pollution.parameter};{p}\n")

In [ ]:
def prepare_traces_dl(x, y, x_att, y_att):
    """
    Normalizes the traces, one-hot encodes the labels.
    Returns profiling traces, labels and attack traces, labels.
    """
    prof_mean, prof_std = x.mean(axis=0), x.std(axis=0)
    norm_x = (x - prof_mean) / prof_std
    norm_x_att = (x_att - prof_mean) / prof_std

    return norm_x, encode(y), norm_x_att, encode(y_att)


def build_mlp(x, y, params):
    mdl = Sequential()
    mdl.add(Dense(100, activation=params['activation'], input_shape=(x.shape[1],)))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(9, activation='softmax'))

    mdl.compile(optimizer=params['optimizer'], loss=params['losses'], metrics=['accuracy'])

    out = mdl.fit(x, y, shuffle=True, batch_size=params['batch_size'], epochs=params['epochs'], verbose=False)

    return out, mdl


def make_mlp(x, y):
    return build_mlp(x, y, {
        'activation': 'relu',
        'optimizer': Adam(learning_rate=0.001),
        'losses': 'categorical_crossentropy',
        'batch_size': 150,
        'epochs': 5
    })[1]

def wegener_p(mdl: Model, x_att: np.ndarray, y_att: np.ndarray):
    predictions = mdl.predict(x_att).argmax(axis=1)
    labels = y_att.argmax(axis=1)

    correct = np.sum(predictions == labels)
    total = len(predictions)
    # print(total, correct)

    return binomial_test(total, correct)

LIMIT_PROF = 2000
LIMIT_ATT = 500

def la_benchmark(db: Database, pollution_type: PollutionType, params):
    for param in params:
        pollution = Pollution(pollution_type, param)
        print("Load traces         ", end="\r")
        trace_set = TraceSetHW(db, pollution, (LIMIT_PROF, LIMIT_ATT))

        if os.path.exists(trace_set.path):
            print("Prepare traces (1/2)", end="\r")
            x9, y9, x9_att, y9_att = prepare_traces_dl(*trace_set.profile(), *trace_set.attack())
            print("Prepare traces (2/2)", end="\r")
            (x2, y2), (x2_att, y2_att) = reduce_fixed_fixed(x9, y9), reduce_fixed_fixed(x9_att, y9_att)

            print("Make model (1/2)    ", end="\r")
            mdl9 = make_mlp(x9, y9)
            print("Make model (2/2)    ", end="\r")
            mdl2 = make_mlp_wegener(x2, y2, False)

            print("TVLA                ", end="\r")
            tvla_ps = np.min(tvla_cri(trace_set, ORDER), axis=1)

            print("Predict             ", end="\r")
            dlla9_p, _ = dlla_hw(mdl9, x9_att, y9_att)
            dlla2_p = wegener_p(mdl2, x2_att, y2_att)

            print(f"{pollution_type} ({param}). TVLA ({tvla_ps}). DLLA9 ({dlla9_p}). DLLA2 ({dlla2_p})", end="\r")

            for order, p in enumerate(tvla_ps):
                if order > 0:
                    store_results(db, f"cri_tvla_{order}", pollution, p)

            store_results(db, "dlla9", pollution, dlla9_p)
            store_results(db, "dlla2", pollution, dlla2_p)

        print()

while True:
    # la_benchmark(DB, PollutionType.jitter, JITTER_PARAMS)
    la_benchmark(DB, PollutionType.delay, DELAY_PARAMS)
    la_benchmark(DB, PollutionType.gauss, GAUSS_PARAMS)


PollutionType.delay (0.0). TVLA ([1.00000000e+000 1.08517514e-190 6.18406187e-112]). DLLA9 (1.5968418368655447e-44). DLLA2 (8.983673285347708e-17)
PollutionType.delay (0.05). TVLA ([1.00000000e+000 1.59589737e-123 1.26478509e-095]). DLLA9 (5.998952273735925e-47). DLLA2 (2.4846066690952784e-17)
PollutionType.delay (0.1). TVLA ([1.00000000e+000 4.23877071e-127 7.75615822e-076]). DLLA9 (4.556266626614053e-52). DLLA2 (9.12316211331641e-22)
PollutionType.delay (0.15000000000000002). TVLA ([1.00000000e+000 7.53923606e-125 4.50411260e-091]). DLLA9 (3.5351896388337706e-40). DLLA2 (1.0963855666219223e-15)
PollutionType.delay (0.2). TVLA ([1.00000000e+000 2.10444059e-119 1.07069306e-082]). DLLA9 (4.0722844571366125e-51). DLLA2 (8.983673285347708e-17)
PollutionType.delay (0.25). TVLA ([1.00000000e+000 3.79842040e-123 2.13735468e-076]). DLLA9 (9.955721330321174e-48). DLLA2 (1.0963855666219223e-15)
PollutionType.delay (0.30000000000000004). TVLA ([1.00000000e+000 5.90418415e-112 4.93315553e-081]). 